## clean_economic_index_data

1. reads in economic index csv file
2. creates a year_month column for using as a key to join with other tables
3. saves data as a table in aws postgres database


In [19]:
import pandas as pd



In [20]:
filename='/home/jieliang/proj3/submission/data/economic_index.csv'
    
df = pd.read_csv(filename)
    

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 2 columns):
DATE         332 non-null object
LOIAGRIDX    332 non-null float64
dtypes: float64(1), object(1)
memory usage: 5.3+ KB


In [22]:
df

,DATE,LOIAGRIDX
0,1990-02-01,5.18
1,1990-03-01,3.38
2,1990-04-01,0.18
3,1990-05-01,1.60
4,1990-06-01,2.71
5,1990-07-01,4.66
6,1990-08-01,3.25
7,1990-09-01,0.08
8,1990-10-01,0.31
9,1990-11-01,-4.08


In [23]:
df.DATE=df.DATE.apply(pd.to_datetime)

In [24]:
df['month']=df.DATE.dt.month

In [25]:
df['year']=df.DATE.dt.year

In [26]:
df.head()

,DATE,LOIAGRIDX,month,year
0,1990-02-01,5.18,2,1990
1,1990-03-01,3.38,3,1990
2,1990-04-01,0.18,4,1990
3,1990-05-01,1.60,5,1990
4,1990-06-01,2.71,6,1990


In [27]:
df.month.value_counts()

9     28
8     28
7     28
6     28
5     28
4     28
3     28
2     28
12    27
11    27
10    27
1     27
Name: month, dtype: int64

In [28]:
def to_year_month_string(year,month):
    y='y'
    m=''
    
    if month<10:
        m = '0'+str(month)
    else:
        m = str(month) 
        
    return y+'-'+'m'
        
df['year_month_index'] = df.apply(lambda row: str(row.year)+'-' + str(row.month), axis=1)
    


In [29]:
#df['year_month']

In [30]:
pd.Period('2011-1')

Period('2011-01', 'M')

In [31]:
df.head()

,DATE,LOIAGRIDX,month,year,year_month_index
0,1990-02-01,5.18,2,1990,1990-2
1,1990-03-01,3.38,3,1990,1990-3
2,1990-04-01,0.18,4,1990,1990-4
3,1990-05-01,1.60,5,1990,1990-5
4,1990-06-01,2.71,6,1990,1990-6


In [33]:
df.LOIAGRIDX.value_co()

DATE                332
LOIAGRIDX           274
month                12
year                 28
year_month_index    332
dtype: int64

In [32]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

from sqlalchemy import create_engine
import psycopg2 
import io


connection_args = {
    'host': '54.200.221.80', # current aws instance public IP
    'user': 'ubuntu',    
    'dbname': 'proj3',   
    'port': 5432         
}


connection = pg.connect(**connection_args)


In [15]:
cur = connection.cursor()

cur.execute("""CREATE TABLE economic_index (
  DATE                      TEXT,
  LOIAGRIDX                 REAL,
  month                     INT,
  year                      INT,
  year_month_index                TEXT
  
);""")


In [16]:
output = io.StringIO()
df.to_csv(output, sep='\t', header=False, index=False)
output.seek(0)
contents = output.getvalue()
cur.copy_from(output, 'economic_index', null="") #null values become ''   
connection.commit()

In [17]:
df.shape

(332, 5)

In [18]:
#SELECT expenses.amount, economic_index.LOIAGRIDX FROM expenses, economic_index WHERE expenses.year_month = economic_index.year_month;

